     Trong nội dung này, chúng ta sẽ đưa ra các kết quả thực nghiệm để trả lời các câu hỏi sau:
    

    1. Với các tiêu chí tinh chỉnh mô hình khác nhau thì tác động sẽ như thế nào đến lợi nhuận?
    2.Do có sự đánh đổi giữa các tiêu chí khi ngưỡng được lựa chọn cho phân loại thay đổi thì lợi nhuận thay đổi ra sao?

    Nó quan trong vì các câu hỏi trên sẽ là căn cứ để người làm dữ liệu xây dựng và đưa ra một mô hình tốt nhất để thỏa mãn yêu cầu tiên quyết của công ty là: LỢI NHUẬN chứ không phải các yếu tố yêu thích của người làm về dữ liệu và thống kê là ROC-AUC, F1 SCORE hay RECALL.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
df = pd.read_csv('X_bank.csv')

In [ ]:
df1 = df.drop(['ZIP Code','ID'], axis=1)

In [ ]:
Y = df1'Personal Loan']
X = df1.drop(['Personal Loan'], axis=1)

Chia train test theo tỷ lệ 60-40

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=29, stratify=Y)

Xây dựng prameters grid cho Random Forest (RF):

In [ ]:
RF = RandomForestClassifier()
paramGrid = {'max_depth':[2,3,None],
             'n_estimators':[500,700,1000],
             'max_features':[3,5,10],
             'min_samples_split':[3,5,10],
             'min_samples_leaf':[3,5,10],
             'bootstrap':[True, False],
             'criterion':['gini','entropy']}

Xây dụng 1 function để tùy chỉnh tham số cho RF theo tiêu chí được lựa chọn theo Random Search:

In [ ]:
#hàm tùy chỉnh và train theo 1 tiêu chí chọn trước
def turning_RF(metric_selected):
    RF_Random = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=29),
                                  param_distributions = param_grid,
                                  cv=5,
                                  n_iter=48,
                                  scoring=metric_selected,
                                  n_jobs=-1,
                                  random_state=29)
    RF_Random.fit(X_train, y_train)
    return(rf_random)

Chúng ta sẽ train và tùy chỉnh mô hình RF theo các tiêu chí là accuracy, f1 score, ROC-AUC

In [ ]:
import time
start_time = time.time()
RF_ACCURACY = turning_RF(metric_selected='accuracy')
RF_RECALL = turning_RF(metric_selected='recall')
RF_ROC_AUC = turning_RF(metric_selected='roc_auc')

training_time = time.time() - start_time #thời gian để train
print('Thời gian tinh chỉnh: ', training_time)

Sau khi train, chúng ta có thể có được các giá trị tốt nhất của một tham số theo từng tiêu chí được chọn, bằng cách:

In [ ]:
print(f'Tham số tốt nhất với tiêu chí Accuracy: {RF_ACCURACY.best_params_}')
print(f'Tham số tốt nhất với tiêu chí Recall: {RF_RECALL.best_params_}')
print(f'Tham số tốt nhất với tiêu chí ROC_AUC: {RF_ROC_AUC.best_params_}')

các mô hình RF tối ưu tương ứng với tiêu chí tùy chọn:

In [ ]:
BEST_RF_ACCURACY = RF_ACCURACY.best_estimator_
BEST_RF_RECALL = RF_RECALL.best_estimator_
BEST_RF_ROC = RE_ROC_AUC.best_estimator_

In [ ]:
#Sử dụng RF để dự báo cho nhãn 0-1 cho các cases
PREDICT_ACCURACY = BEST_RF_ACCURACY.predict(X_test)
PREDICT_RECALL = BEST_RF_RECALL.predict(X_test)
PREDICT_ROC = BEST_RF_ROC_AUC.predict(X_test)

Chúng ta có thể có được probability of Default thu được từ RF, bằng cách:

In [ ]:
PREDICT_PROB_ACCURACY  = BEST_RF_ACCURACY.predict_proba(X_test)
PREDICT_PROB_RECALL = BEST_RF_RECALL.predict_proba(X_test)
PREDICT_PROB_ROC = BEST_RF_ROC.predict_proba(X_test)

    Sự thay đổi của profit khi probability of cutoff thay đổi:

Một khách hàng sẽ được phân loại là tốt hay xấu phụ thuộc vào hai yếu tố: (1) xác suất (probability of default) dự báo từ mô hình, và (2) ngưỡng cutoff được lựa chọn để phân loại khách hàng dựa trên probability of default thu được từ mô hình.

(1) Nếu mỗi một khách hàng được đồng ý cho vay bởi ngân hàng thì KH sẽ được giải ngân một khoản vay đúng bằng thu nhập của họ (thực tế các ngân hàng Việt Nam khi cấp tín dụng tiêu dùng, nếu khách hàng thỏa mãn thì họ sẽ được cấp một khoản vay thường là gấp nhiều lần thu nhập sau thuế của họ), và (2) Một khách hàng là xấu (có khả năng không thanh toán được nợ và vỡ nợ) nhưng RF phân loại nhầm thành tốt thì ngân hàng sẽ mất trắng khoản vay này.

Chúng ta có thể thêm vào 1 function để tính số lãi thu về của ngân hàng - trong trường hợp này, chúng ta chọn 10%

In [ ]:
def profit_by_cutoff(cutoff, pred_prob):
    #phân loại là 0 hay 1 phụ thuộc vào probability of default và ngưỡng cutoff được chọn:
    PREDICT_BG = (pred_prob[:,1] > cutoff).astype(int)
    #các trường hợp là 0 được phân loại đúng thành 0
    G = X_test[(y_test == 0)&(pred_bg == 0)]
    #các trường hợp là 1 nhưng được phân loại sai thành 0
    B = X_test[(y_test == 1)&(pred_bg == 0)]
    #Tính profit
    profit = np.sum(0.1*G['Income']) - np.sum(B['Income'])
    
    return(profit)

Với lãi là 10% thì profit cho mô hình RF cho các tiêu chí. Cũng như có thể thấy được là RF tùy chỉnh theo tiêu chí nào sẽ có lợi nhuận cao nhất với ngưỡng cutoff là 0.5!!

In [ ]:
PROB = [PREDICT_PROB_ACCURACY, PREDICT_PROB_RECALL, PREDICT_PROB_ROC]
CRITERIONS = ['ACCURACY','RECALL','ROC']

for P, C in zip(PROB, CRITERIONS):
    print(f'The profit by {C} is {profit_by_cutoff(cutoff=0.5, pred_prob=P)}')

Chúng ta có thể thêm vào 1 function để do lường profit thay đổi như thế nào nếu như ngưỡng cutoff thay đổi.

In [ ]:
def profit_cut_off_range(prob, cut_off_range):
    profit = []
    for i in cut_off_range:
        profit_i = profit_by_cutoff(cutoff=i, pred_prob = prob)
        profit.append(profit_i)
    
    df_results = pd.DataFrame({'CutOff': np.round(cut_off_range, 4), 'Profit': profit})
    return df_results

In [ ]:
Có thể lựa chọn các ngưỡng cutoff khác nhau!

In [ ]:
cut_off_range_profit = np.arange(.025, .60, .65, .70)

In [ ]:
# Tính toán Profit dựa trên các kịch bản khác nhau của cutoff
FROFIT_ROC = profit_cut_off_range(prob=PREDICT_PROB_ROC, cut_off_range=cut_off_range_profit)
FROFIT_RECALL  = profit_cut_off_range(prob=PREDICT_PROB_RECALL, cut_off_range=cut_off_range_profit)
FROFIT_ACCURACY = profit_cut_off_range(prob=PREDICT_PROB_ACCURACY, cut_off_range=cut_off_range_profit)

Profit dưới các kịch bản khách nhau của cutoff và tiêu chí được lựa chọn cho tinh chỉnh RF:

In [ ]:
plt.figure(figsize=(9,6))
plt.plot('CutOff','Profit', data = FROFIT_ROC, label = 'AUC-ROC', lw=2)
plt.plot('CutOff','Profit', data = FROFIT_RECALL, label = 'Recall', lw=2)
plt.plot('CutOff','Profit', data = FROFIT_ACCURACY, label = 'Accuracy', lw=2)

plt.scatter('CutOff','Profit', data = FROFIT_ROC[FROFIT_ROC['CutOff'] == 0.5], marker = 'D', s=60, label = None)
plt.scatter('CutOff','Profit', data = FROFIT_RECALL[FROFIT_RECALL['CutOff'] == 0.5], marker = 'D', s=60, label = None)
plt.scatter('CutOff','Profit', data = FROFIT_ACCURACY[FROFIT_ACCURACY['CutOff'] == 0.5], marker = 'D', s=60, label = None)


plt.xlabel('Cutoff')
plt.ylabel('Profit')
plt.title('Profit by cutoff and Method of turning Random Forest \n')
plt.legend(loc='best')
plt.show()

Chúng ta có thể minh họa lợi nhuận tương ứng với kịch bản tốt nhất của cutoff cho các mô hình RF được tinh chỉnh theo các tiêu chí khác nhau như sau:

In [ ]:
PROFIT_ROC_MAX = FROFIT_ROC[FROFIT_ROC['CutOff'] == df_profit_roc['CutOff'].max()]
PROFIT_RECALL_MAX = data = FROFIT_RECALL[FROFIT_RECALL['CutOff'] == df_profit_rec['CutOff'].max()]
PROFIT_ACCURACY_MAX = FROFIT_ACCURACY[FROFIT_ACCURACY['CutOff'] == df_profit_acc['CutOff'].max()]

In [ ]:
#Plot
plt.figure(figsize=(9,6))
plt.plot('CutOff','Profit', data = FROFIT_ROC, label = 'AUC-ROC', lw=2)
plt.plot('CutOff','Profit', data = FROFIT_RECALL, label = 'Recall', lw=2)
plt.plot('CutOff','Profit', data = FROFIT_ACCURACY, label = 'Accuracy', lw=2)

plt.scatter('CutOff','Profit', data = PROFIT_ROC_MAX, marker = 'D', s=70, label = None)
plt.scatter('CutOff','Profit', data = PROFIT_RECALL_MAX, marker = 'D', s=70, label = None)
plt.scatter('CutOff','Profit', data = PROFIT_ACCURACY_MAX, marker = 'D', s=70, label = None)


plt.xlabel('Cutoff')
plt.ylabel('Profit')
plt.title('Figure 2: Profit by cutoff and Method of turning Random Forest \n')
plt.legend(loc='best')
plt.show()

    Sử dụng logistic regression để khảo sát lợi nhuận
    

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

In [ ]:
#sử dụng logistic regression để đánh giá probability of default
PREDICT_PROB_LOGIT = logmodel.predict_proba(X_test)

In [ ]:
#các kịch bản về lợi nhuận nếu chúng ta sử dụng mô hình logistic
PROFIT_LOGIT = profit_cut_off_range(prob = PREDICT_PROB_LOGIT, cut_off_range = cut_off_range_profit)

In [ ]:
#option có lợi nhuận tốt nhất với logistic regression
PROFIT_LOGIT_MAX = PROFIT_LOGIT[PROFIT_LOGIT ['Profit'] == PROFIT_LOGIT['Profit'].max()]
print(f'The maximum profit: {PROFIT_LOGIT_MAX}')

Chênh lệch về profit giữa 2 mô hình RF và logistic regression

In [ ]:
np.array(max_profit_roc['Profit'])/np.array(max_profit_logit['Profit']) - 1

Chúng ta có thể  plot các kịch bản khác nhau về profit theo cutoff

In [ ]:
plt.figure(figsize=(9,6))
plt.plot('CutOff','Profit', data = PROFIT_LOGIT, lw=2)
plt.scatter('CutOff','Profit', data = PROFIT_LOGIT_MAX, color='red', s=70, marker='D')
plt.scatter('CutOff','Profit', data = PROFIT_LOGIT[PROFIT_LOGIT['CutOff'] ==0.5], marker = 'x', s=70, color='purple', label=None)
plt.xlabel('Cutoff')
plt.ylabel('Profit')
plt.title('Figure 3: Profit by Cutoff for Logistic Regression')
plt.show()

    Sự thay đổi của ACCURACY theo ngưỡng cuffoff được chọn

In [ ]:
def accuracy_cut_off_range(prob_pred, cut_off_range_n):
    accuracy = []
    
    for j in cut_off_range_n:
        PREDICT_BG = (prob_pred[:,1] >=j).astype(int)
        acc_i = np.mean(PREDICT_BG == y_test)
        accuracy.append(acc_i)
        
    DF_ACCURACY = pd.DataFrame({'cut_off': np.round(cut_off_range_n,4), 'accuracy':accuracy})
    return(df_accuracy)


In [ ]:
#rang của cutoff
cut_off_range_acc = np.arange(0.025,1,0.025)

In [ ]:
#ACCURACY tương ứng
ACCURACY_RF_ACC = accuracy_cut_off_range(prob_pred = PREDICT_PROB_ACCURACY, cut_off_range_n = cut_off_range_acc)
ACCURACY_RF_RECALL = accuracy_cut_off_range(prob_pred = PREDICT_PROB_RECALL, cut_off_range_n = cut_off_range_acc)
ACCURACY_RF_ROC = accuracy_cut_off_range(prob_pred = PREDICT_PROB_ROC, cut_off_range_n = cut_off_range_acc)
ACCURACY_LG_ACC = accuracy_cut_off_range(prob_pred = PREDICT_PROB_LOGIT, cut_off_range_n = cut_off_range_acc)

In [ ]:
# Biến đổi của Accuracy theo cutoff: 
plt.figure(figsize=(9,6))
plt.plot('cut_off','accuracy', data = ACCURACY_RF_ACC, label = 'Accuracy', lw=2)
plt.plot('cut_off','accuracy', data = ACCURACY_RF_RECALL, label = 'Recall', lw=2)
plt.plot('cut_off','accuracy', data = ACCURACY_RF_ROC , label = 'ROC-AUC', lw=2)
plt.plot('cut_off','accuracy', data = ACCURACY_LG_ACC, label = 'Logistic Regression', lw=2)

plt.xlabel('Cutoff')
plt.ylabel('Accuracy')
plt.title('Figure 4: Trade off between Accuracy and Cutoff')
plt.legend()
plt.show()

    KẾT LUẬN

Dựa trên các kết quả thực nghiệm thu được,chúng ta thấy rằng với bài toán phân loại và xếp hạng hồ sơ xin vay thì tiêu chí phù hợp để tinh chỉnh và lựa chọn mô hình trước hết phải căn cứ vào mục tiêu kinh doanh của tổ chức là lợi nhuận. Điều này có nghĩa là, không phải AUC/ROC, Accuracy, Recall hay bất cứ tiêu chí nào khác được chọn để tinh chỉnh cũng như so sánh - lựa chọn mô hình cho mục đích sử dụng cuối cùng mà là lợi nhuận.